# Image Classification

Satellite images often need to be classified (assigned to a fixed set of types) or to be used for detection of various features of interest.  Here we will look at the classification case, using labelled satellite images from various categories from the [UCMerced LandUse dataset](http://weegee.vision.ucmerced.edu/datasets/landuse.html). scikit-learn is useful for general numeric data types, but it doesn't have significant support for working with images. Luckily, there are various deep-learning and convolutional-network libraries that do support images well, including Keras (backed by TensorFlow) as we will use here. To run this notebook, you will first need to download the dataset and put it in ../data/.

<!-- Direct link: http://weegee.vision.ucmerced.edu/datasets/UCMerced_LandUse.zip -->

In [ ]:
import os
import glob

import numpy as np
import geoviews as gv
import holoviews as hv
import pandas as pd

gv.extension('bokeh')

#### Get the classes and files

The files in the dataset are grouped into folders by their category. We will extract the categories from the directory names and then create a list of files for each category:

In [ ]:
path = '../data/UCMerced_LandUse/Images/'
classes = np.array([f.split(os.sep)[-1] for f in glob.glob(path+'*')])
files = {c: glob.glob(os.path.join(path, c, '*')) for c in classes}

#### Split files into train and test sets

In order to accurately test the performance of the classifier we are building we will split the data into training and test sets with an 80/20 split. We randomly sample the files in each category and assign them either to the training or test set:

In [ ]:
train_set = list(np.random.choice(np.arange(100), 80, False))
test_set = [i for i in range(100) if i not in train_set]

train_files = {c: [f for f in fs if int(f[-6:-4]) in train_set] for c, fs in files.items()}
test_files  = {c: [f for f in fs if int(f[-6:-4]) in test_set]  for c, fs in files.items()}

#### Define function to sample from train or test set

Next we define a function that randomly samples an image, either from the training or test set:

In [ ]:
def get_sample(cls, set='training'):
    files = train_files if set == 'training' else test_files
    flist = list(files[cls])
    f = flist[np.random.randint(len(flist))]
    return gv.RGB.load_tiff(f).relabel(cls)

We can inspect the data on one of these samples to see that the data is loaded as an xarray Dataset:

In [ ]:
get_sample('forest').data

Each sample is returned as an holoviews RGB image so we can visualize it:

In [ ]:
gv.Layout([get_sample(s).options(xaxis=None, yaxis=None) for s in np.random.choice(classes, 4)]).cols(2)

## Define the model

Now it's time to define a model, the code below The code snippet below defines a convolutional neural network containing a stack of 3 convolution layers with a ReLU activation and followed by max-pooling layers. This is very similar to the architectures that Yann LeCun advocated in the 1990s for image classification (with the exception of ReLU).

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

size = (150, 150)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(*size, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(21))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Declare the data

The dataset of images we have is relatively small so to avoid overfitting we will want to apply some augmentation to it. The code below defines a generator that randomly samples from either the training or test set and generates a randomly rotated and cropped 150x150 window onto the 256x256 images along with a label array:

In [ ]:
def get_array(rgb, size=(150, 150)):
    array = np.dstack([rgb.dimension_values(d, flat=False) for d in rgb.vdims])
    # Randomly flip
    if np.random.rand() >= 0.5:
        array = array.transpose(1, 0, 2)
    if np.random.rand() >= 0.5:
        array = array[::-1]
    if np.random.rand() >= 0.5:
        array = array[:, ::-1]
    # Randomly crop
    sh, sw = size
    h, w = array.shape[:2]
    b = np.random.randint(h-sh)
    l = np.random.randint(w-sw)
    array = array[b:b+sh, l:l+sw]
    return array/255.
    
class_list = list(classes)

def gen_samples(set='training', labels=None):
    "Generates random arrays along with class labels"
    while True:
        choice = np.random.choice(classes)
        if labels is not None:
            labels.append(choice)
        one_hot = np.zeros((21,))
        one_hot[class_list.index(choice)] = 1
        yield get_array(get_sample(choice, set))[np.newaxis, :], one_hot[np.newaxis, :]        

## Run the model

Before we start running the model let's set up a keras Callback to build a dashboard to monitor the accuracy and loss during training:

In [ ]:
import time
from keras.callbacks import Callback

class MonitorCallback(Callback):
    """
    Builds a streaming dashboard to monitor the accuracy
    and loss during training using HoloViews streams.
    """
    
    _format = '%s - Epoch: %d - Elapsed time: %.2fs'
    
    def __init__(self, metrics=['acc', 'loss']):
        super().__init__()
        sample = {'Epoch': np.array([])}
        for metric in metrics:
            sample[metric] = np.array([])
        self.buffer = hv.streams.Buffer(sample)
        dmaps = []
        for metric in metrics:
            def cb(data, metric=metric):
                return hv.Curve(
                    data, 'Epoch', metric, label=self._format
                    % (metric, self.epoch, self.elapsed_time))
            dmap = hv.DynamicMap(cb, streams=[self.buffer])
            dmaps.append(dmap)
        self.layout = hv.Layout(dmaps)
        self.metrics = metrics
        self.start_time = None
        self.epoch = 0

    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        
    @property
    def elapsed_time(self):
        if self.start_time is None:
            return 0
        else:
            return time.time() - self.start_time

    def on_epoch_end(self, epoch, logs=None):
        self.epoch += 1
        data = {'Epoch': [self.epoch]}
        for metric in self.metrics:
            data[metric] = [logs.get(metric)]
        self.buffer.send(data)

Now we can create an instance of the callback and display the dashboard, which will at first appear blank.

In [ ]:
monitor = MonitorCallback()
monitor.layout.options('Curve', width=450)

Next we will fit the model with our training data generator, as the model is running it will update the dashboard above:

In [ ]:
%%time
history = model.fit_generator(gen_samples('training'), steps_per_epoch=50, epochs=500, verbose=False, callbacks=[monitor])

## Evaluate the model

First we will have a look at the monitoring output but smooth it slightly so we can make out the overall trend:

In [ ]:
from holoviews.operation.timeseries import rolling
monitor.layout.options('Curve', width=400).map(rolling, hv.Curve)

Now let us test the predictions on the test set, first visually:

In [ ]:
def get_prediction(cls):
    sample = get_sample(cls, 'test')
    array = get_array(sample)[np.newaxis, ...]
    p = model.predict(array).argmax()
    p = classes[p]
    return sample.relabel('Predicted: %s - Actual: %s' % (p, cls))

opts = dict(fontsize={'title': '8pt'}, xaxis=None, yaxis=None, width=250, height=250)
hv.Layout([get_prediction(cls).options(**opts) for cls in classes[:20]]).cols(3)

And now numerically by running 1,000 predictions on the test set:

In [ ]:
ntesting = 1000
labels = []
test_gen = gen_samples('test', labels)
prediction = model.predict_generator(test_gen, steps=ntesting)
y_pred = classes[prediction.argmax(axis=1)]
y_true = np.array(labels[:ntesting])

accuracy = (y_pred==y_true).sum()/ntesting

print(f'Accuracy on test set {accuracy}')

Next we can see how well the classifier performs on the different categories. To see how it performs we will make 50 predictions on each category and record both the accuracy and the predictions:

In [ ]:
def predict(cls, iterations=50):
    accurate, predictions = [], []
    for i in range(iterations):
        sample = get_sample(cls, 'test')
        array = get_array(sample)[np.newaxis, ...]
        p = model.predict(array).argmax()
        p = classes[p]
        predictions.append(p)
        accurate.append(p == cls)
    return np.sum(accurate)/float(iterations), predictions

accuracies = [(c, *predict(c)) for c in classes]

We can now break down the accuracy by landuse category:

In [ ]:
df = pd.DataFrame(accuracies, columns=['landuse', 'accuracy', 'predictions'])

hv.Bars(df, 'landuse', 'accuracy', label='Accuracy by Landuse Category').options(
    width=700, xrotation=45, color='landuse', 
    cmap='Category20')

Another interesting way of viewing this data is to look at which categories the classifier got confused on. We will count how many times the classifier classified one category as another category and visualize the result as a Chord graph where each edge is colored by the actual category. By clicking on a node we can reveal which other categories incorrectly identified an image as being of that category:

In [ ]:
pdf = pd.DataFrame([(p, l) for (_, l, _, ps) in df.itertuples() for p in ps], columns=['Prediction', 'Actual'])
graph = pdf.groupby(['Prediction', 'Actual']).size().to_frame().reset_index()
confusion = graph.rename(columns={0: 'Count'})

hv.Chord(confusion).relabel('Confusion Graph').options(
    node_color='index', cmap='Category20', edge_color='Actual', labels='index',
    width=600, height=600)

Clicking on buildings, for instance, reveals a lot of confusion about overpasses, mediumresidential, and intersections, all of which do share visual features in common. Conversely, number of buildings were misidentified as parklots, which is also reasonable. As we saw in the bar chart above, forests on the other hand, have lots of edges leading back to itself, demonstrating the high accuracy observed for that category of images.

In [ ]:
confusion.Count /= 50
hv.HeatMap(confusion, label='Confusion Matrix').sort().options(
    xrotation=45, width=500, height=500, cmap='viridis', tools=['hover'], invert_yaxis=True)